<a href="https://colab.research.google.com/github/jsqzhu/nbme/blob/main/02_NSMB_Bert_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 37.2 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.backend as K
from tensorflow.keras import mixed_precision
#import tensorflow_addons as tfa
#import dill
import matplotlib.pyplot as plt
import transformers
from transformers import AutoTokenizer, AutoConfig,TFAutoModel
import json
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('https://raw.githubusercontent.com/jsqzhu/nbme/main/data.csv')
train.head()

,id,case_num,pn_num,feature_num,annotation,location,pn_history,feature_text,annot_count,loc_count
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-MI-OR-Family-history-of-myoc...,1,1
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-thyroid-disorder,1,1
2,00016_002,0,16,2,['chest pressure'],['203 217'],HPI: 17yo M presents with palpitations. Patien...,Chest-pressure,1,1
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",HPI: 17yo M presents with palpitations. Patien...,Intermittent-symptoms,2,2
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258'],HPI: 17yo M presents with palpitations. Patien...,Lightheaded,1,1


# Functions

https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT

# Prepare Training Dataset

## Prepare feature entities

In [ ]:
print('# unique feature labels: ', train['feature_num'].nunique())

# unique feature labels:  143


In [ ]:
train['feature_num'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
       100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 300, 301, 302, 303, 304, 305, 306, 307, 308,
       309, 310, 311, 312, 313, 314, 315, 400, 401, 402, 403, 404, 405,
       406, 407, 408, 409, 500, 501, 502, 503, 504, 505, 506, 507, 508,
       509, 510, 511, 512, 513, 514, 515, 516, 517, 600, 601, 602, 603,
       604, 605, 606, 607, 608, 609, 610, 611, 700, 701, 702, 703, 704,
       705, 706, 707, 708, 800, 801, 802, 803, 804, 805, 806, 807, 808,
       809, 810, 811, 812, 813, 814, 815, 816, 817, 900, 901, 902, 903,
       904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916])

Recall that 4399 cases have no corresponding text in pn_history for the target feature. Thus, we need to create an additional class to account for the texts with no annotations

In [ ]:
Label = ['Empty',] + train['feature_num'].unique().tolist() 
le = LabelEncoder()
le.fit(Label)

LabelEncoder()

In [ ]:
train['target'] = le.transform(train['feature_num'])

## Prepare Model Input

Use transformer with TF2.0: https://www.kdnuggets.com/2021/02/hugging-face-transformer-basics.html#:~:text=NLP%2Dfocused%20startup%20Hugging%20Face,for%20training%20and%20evaluation%20purposes.

In [ ]:
# load model
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
test_text = train['pn_history'][0]

https://huggingface.co/docs/transformers/main_classes/tokenizer

In [ ]:
text_processed = tokenizer(test_text, 
                           max_length=512,
                           padding='max_length',
                           truncation=True,
                           return_offsets_mapping=True)

In [ ]:
print(f'text: \'{test_text}\'')
print(f'input ids: {text_processed["input_ids"]}')
print(f'attention mask: {text_processed["attention_mask"]}')
print(f'offset_mapping: {text_processed["offset_mapping"]}')

text: 'HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of "heart beating/pounding out of my chest." 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. 
PMHx: none
Rx: uses friends adderrall
FHx: mom with "thyroid disease," dad with recent heart attcak
All: none
Immunizations: up to date
SHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms'
input ids: [101, 6857, 1182, 131, 1542, 7490, 182, 8218, 11

In [ ]:
# need to check the max length
# check if len(subtoken) longer than max_length from the model?
https://ai.stackexchange.com/questions/37624/why-do-transformers-have-a-fixed-input-length
# how to handle longer tokens
https://stackoverflow.com/questions/74290497/how-to-handle-sequences-longer-than-512-tokens-in-layoutlmv3
# for offset_mapping
# https://huggingface.co/transformers/v4.2.2/custom_datasets.html

use model tokenizer to prepare input text; 
convert pn_history to word embedding as input features 

## Prepare Model Output

map each pn_history using feature entities and location

In [ ]:
#create target output

#mylist = literal_eval(mylist)

In [ ]:
#make sure the sequence lenth doesn't exceed 512

## Prepare dataset